## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH09-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH09-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한반도 동쪽에 위치한 대한민국은 수도는 서울이며 인구는 약 5천만 명이다. 경제는 선진국으로 발전하여 세계에서 세 번째로 큰 경제체제를 갖고 있다. 한국은 전통적인 문화와 현대화가 조화를 이루는 나라로, 한류 문화로 유명하다. 또한 기술력이 뛰어나 IT 분야에서 선두를 달리는 나라이며, K-pop과 영화 등 엔터테인먼트 산업도 세계적으로 유명하다. 한국은 고속철도와 지하철 등 교통 인프라가 잘 구축되어 있고, 자연 경관과 역사적인 유적지도 풍부하다. 현재 분단 상태인 한반도에서 남쪽에 위치한 대한민국은 북한과의 관계와 통일 문제에 대한 과제를 안고 있다.
CPU times: user 74.2 ms, sys: 28.7 ms, total: 103 ms
Wall time: 6.71 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [5]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 북한으로 나뉘는 국가이다. 대한민국은 수도가 서울에 위치하며 민주주의 체제를 따르고 있고 경제적으로 선진국으로 발전하였다. 한국은 전통적인 문화와 현대적인 발전이 공존하는 나라로, K-pop, K-drama, 한국 요리 등이 전 세계적으로 인기를 끌고 있다. 또한 한반도 북쪽에 위치한 북한은 공산주의 체제를 가지고 있으며 국제사회와의 관계가 긴장되어 있는 상태이다. 한국은 뛰어난 기술력과 산업력을 갖추고 있어 세계적인 기업들을 배출하고 있으며, 한류 열풍으로 한국 문화가 전 세계에 널리 알려져 있다.
CPU times: user 292 ms, sys: 57.5 ms, total: 349 ms
Wall time: 3.19 s


In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 북한으로 나뉘는 국가이다. 대한민국은 수도가 서울에 위치하며 민주주의 체제를 따르고 있고 경제적으로 선진국으로 발전하였다. 한국은 전통적인 문화와 현대적인 발전이 공존하는 나라로, K-pop, K-drama, 한국 요리 등이 전 세계적으로 인기를 끌고 있다. 또한 한반도 북쪽에 위치한 북한은 공산주의 체제를 가지고 있으며 국제사회와의 관계가 긴장되어 있는 상태이다. 한국은 뛰어난 기술력과 산업력을 갖추고 있어 세계적인 기업들을 배출하고 있으며, 한류 열풍으로 한국 문화가 전 세계에 널리 알려져 있다.
CPU times: user 7.85 ms, sys: 934 μs, total: 8.79 ms
Wall time: 10 ms


## SQLite Cache


In [9]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [10]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response.content)

미국은 북아메리카 대륙의 주요 국가로, 50개의 주로 이루어져 있으며 워싱턴 D.C.를 수도로 갖고 있다. 세계 최대의 경제 규모와 군사력을 보유하고 있으며, 다양한 인종과 문화가 공존하는 다문화국가이다. 미국은 과학기술, 영화, 음악, 문학 등의 분야에서 세계적인 영향력을 행사하고 있으며, 자유주의와 민주주의 원칙을 중시하는 국가로서 국제사회에서도 중요한 역할을 수행하고 있다. 그러나 최근 몇 년간 정치적 분열과 사회적 문제들이 부각되어 있으며, 경제적 불평등과 인종차별 문제 등에 대한 해결이 시급한 과제로 대두되고 있다.
CPU times: user 45.3 ms, sys: 8.11 ms, total: 53.4 ms
Wall time: 2.97 s


In [11]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response.content)

미국은 북아메리카 대륙의 주요 국가로, 50개의 주로 이루어져 있으며 워싱턴 D.C.를 수도로 갖고 있다. 세계 최대의 경제 규모와 군사력을 보유하고 있으며, 다양한 인종과 문화가 공존하는 다문화국가이다. 미국은 과학기술, 영화, 음악, 문학 등의 분야에서 세계적인 영향력을 행사하고 있으며, 자유주의와 민주주의 원칙을 중시하는 국가로서 국제사회에서도 중요한 역할을 수행하고 있다. 그러나 최근 몇 년간 정치적 분열과 사회적 문제들이 부각되어 있으며, 경제적 불평등과 인종차별 문제 등에 대한 해결이 시급한 과제로 대두되고 있다.
CPU times: user 12.9 ms, sys: 3.42 ms, total: 16.3 ms
Wall time: 74.1 ms
